In [1]:
import pandas as pd
import numpy as np

## Importing

In [2]:
dm_df = pd.read_pickle('./dm_drug_class_numbers_encr.pkl')

In [3]:
dm_df

## Building Dominant Drug

In [4]:
dm_df.columns[0:8]

In [5]:
handy = dm_df[
    ['A10A', 'A10BA', 'A10BB', 'A10BD', 'A10BF', 'A10BG', 'A10BJ', 'A10BX']
].copy()
handy

In [6]:
dominant_df = pd.DataFrame(index=dm_df.index)

In [7]:
# Determining insulin
dominant_df.loc[handy['A10A'] > 0, 'insulin'] = True

In [8]:
dominant_df['insulin'].value_counts(dropna=False, normalize=True)

In [9]:
dominant_df

In [11]:
handy.sum(axis=1)

In [10]:
temp = handy.apply(lambda c: c / handy.sum(axis=1).values)
temp

In [20]:
cumsum_df = temp.stack().to_frame().groupby('member').apply(lambda g: g.cumsum())

In [29]:
cumsum_df

### Sandbox

In [35]:
# Specifying dominant drugs
cut_off = 0.8
(cumsum_df > cut_off).head(20)

In [38]:
a = (cumsum_df > cut_off).loc[0]
b = (cumsum_df > cut_off).loc[0].shift(1)

In [40]:
a

In [41]:
b

In [42]:
temp = a & b

In [43]:
temp

In [51]:
' '.join(a.loc[~ temp.values].index.sort_values())

### main

In [116]:
def get_dominant_string(g):
    g = g.droplevel(0, axis=0)
    temp = g & g.shift()
    output = ' '.join(g.loc[~ temp.values].index.sort_values())
    # print(output)
    return output

In [118]:
dominants_dict = dict()

In [119]:
dominants_dict[cut_off] = (cumsum_df > cut_off).groupby(
    level=0
).apply(get_dominant_string)

In [126]:
dominants_dict[0.8]

In [127]:
dominant_df[cut_off] = dominants_dict[cut_off]

In [130]:
dominant_df

In [145]:
dominant_df.loc[dominant_df['insulin'] == True, 0.8].value_counts(normalize=True).head(20)

In [146]:
dominant_df.loc[dominant_df['insulin'].isna(), 0.8].value_counts(normalize=True).head(20)

In [147]:
dominant_df

In [148]:
# Saving
# dominant_df.to_pickle('./results/dominants_0.8.pkl')